In [2]:
import json
import hashlib
import os
from algosdk import mnemonic
from algosdk.v2client import algod
from algosdk.future.transaction import AssetConfigTxn, wait_for_confirmation

In [3]:
import sys
sys.path.append('../scripts')
from create_account import create_account
from closeout_account import closeout_account
from create_nfts import print_created_asset, print_asset_holding

In [13]:
# For ease of reference, add account public and private keys to
# an accounts dict.
print("--------------------------------------------")
print("Creating account...")
accounts = {}
m = create_account()
accounts[1] = {}
accounts[1]['pk'] = mnemonic.to_public_key(m)
accounts[1]['sk'] = mnemonic.to_private_key(m)

--------------------------------------------
Creating account...
My address: I5XCXAH33IQAWY574KUM3R3OZNSDMWIWB5WBZ7FXPHNZCWBBQD5F6LYR4A
Account balance: 0 microAlgos

Go to the below link to fund the created account using testnet faucet: 
 https://dispenser.testnet.aws.algodev.network/?account=I5XCXAH33IQAWY574KUM3R3OZNSDMWIWB5WBZ7FXPHNZCWBBQD5F6LYR4A
Fund transfer in process...
Fund transferred!
Account balance: 5000000 microAlgos



In [4]:

# Change algod_token and algod_address to connect to a different client
algod_token = "2f3203f21e738a1de6110eba6984f9d03e5a95d7a577b34616854064cf2c0e7b"
algod_address = "https://academy-algod.dev.aws.algodev.network/"

algod_client = algod.AlgodClient(algod_token, algod_address)

## Creating Asset

In [21]:
print("--------------------------------------------")
print("Creating Asset...")
# CREATE ASSET
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True
    
# JSON file

f = open ('../data/aliceNFTmetadata.json', "r")

# Reading from file
metadataJSON = json.loads(f.read())
metadataStr = json.dumps(metadataJSON)

hash = hashlib.new("sha512_256")
hash.update(b"arc0003/amj")
hash.update(metadataStr.encode("utf-8"))
json_metadata_hash = hash.digest()



--------------------------------------------
Creating Asset...


In [22]:
# Account 1 creates an asset called latinum and
# sets Account 1 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction
txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    total=1,
    default_frozen=False,
    unit_name="ALICE001",
    asset_name="Alice's Artwork 001",
    manager=accounts[1]['pk'],
    reserve=None,
    freeze=None,
    clawback=None,
    strict_empty_address_check=False,
    url="https://drive.google.com/file/d/1-6ArwT3cmKDOQlOr6AEocSsqIUhjc4Os/view?usp=share_link", 
    metadata_hash=json_metadata_hash,
    decimals=0)

In [25]:
# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])

# Send the transaction to the network and retrieve the txid.
txid = algod_client.send_transaction(stxn)
print("Asset Creation Transaction ID: {}".format(txid))

# Wait for the transaction to be confirmed
confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
print("TXID: ", txid)
print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))


Asset Creation Transaction ID: 4QC3FBP6NK3V5HPEWQQFYU2XGSFACZAHCFCH24E2HCY2MFI4MZ3A
TXID:  4QC3FBP6NK3V5HPEWQQFYU2XGSFACZAHCFCH24E2HCY2MFI4MZ3A
Result confirmed in round: 25439683


In [26]:
try:
    # Pull account info for the creator
    # account_info = algod_client.account_info(accounts[1]['pk'])
    # get asset_id from tx
    # Get the new asset's information from the creator account
    ptx = algod_client.pending_transaction_info(txid)
    asset_id = ptx["asset-index"]
    print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
except Exception as e:
    print(e)

print("--------------------------------------------")
print("You have successfully created your own Non-fungible token! For the purpose of the demo, we will now delete the asset.")
print("Deleting Asset...")

Asset ID: 121850449
{
    "creator": "I5XCXAH33IQAWY574KUM3R3OZNSDMWIWB5WBZ7FXPHNZCWBBQD5F6LYR4A",
    "decimals": 0,
    "default-frozen": false,
    "manager": "I5XCXAH33IQAWY574KUM3R3OZNSDMWIWB5WBZ7FXPHNZCWBBQD5F6LYR4A",
    "metadata-hash": "dtbFu/HZOl7mEmR0an1OrDN4vMo2SQeu1690lqNwzOs=",
    "name": "Alice's Artwork 001",
    "name-b64": "QWxpY2UncyBBcnR3b3JrIDAwMQ==",
    "total": 1,
    "unit-name": "ALICE001",
    "unit-name-b64": "QUxJQ0UwMDE=",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 121850449
{
    "amount": 1,
    "asset-id": 121850449,
    "is-frozen": false
}
--------------------------------------------
You have successfully created your own Non-fungible token! For the purpose of the demo, we will now delete the asset.
Deleting Asset...


In [19]:
# Asset destroy transaction
txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    index=asset_id,
    strict_empty_address_check=False
    )

# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
txid = algod_client.send_transaction(stxn)
print("Asset Destroy Transaction ID: {}".format(txid))

# Wait for the transaction to be confirmed
confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
print("TXID: ", txid)
print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
# Asset was deleted.
try:
    print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
    print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    print("Asset is deleted.")
except Exception as e:
    print(e)

print("--------------------------------------------")
print("Sending closeout transaction back to the testnet dispenser...")
closeout_account(algod_client, accounts[1] )


Asset Destroy Transaction ID: GR4WHDRYV5YD46IUI24GHPLZFR3BFLVIILYKZCBQDHFMCD2K4BXQ
TXID:  GR4WHDRYV5YD46IUI24GHPLZFR3BFLVIILYKZCBQDHFMCD2K4BXQ
Result confirmed in round: 25439606
Asset is deleted.
--------------------------------------------
Sending closeout transaction back to the testnet dispenser...
--------------------------------------------
Closing out account......
Transaction Info:
Signed transaction with txID: Z73TNLLXUADTSYENPJDD3EUNR5WU32RTK2ZZC7XITYHEFQMWRDXA
TXID:  Z73TNLLXUADTSYENPJDD3EUNR5WU32RTK2ZZC7XITYHEFQMWRDXA
Result confirmed in round: 25439609
Account balance: 0 microAlgos

Account Closed


In [21]:
# Change algod_token and algod_address to connect to a different client
algod_token = "2f3203f21e738a1de6110eba6984f9d03e5a95d7a577b34616854064cf2c0e7b"
algod_address = "https://academy-algod.dev.aws.algodev.network/"

algod_client = algod.AlgodClient(algod_token, algod_address)
account_info = algod_client.account_info('I5XCXAH33IQAWY574KUM3R3OZNSDMWIWB5WBZ7FXPHNZCWBBQD5F6LYR4A')
idx = 0
assetid = 121850449
for my_account_info in account_info['assets']:
    scrutinized_asset = account_info['assets'][idx]
    print(account_info)
    idx = idx + 1        
    if (scrutinized_asset['asset-id'] == assetid):
        print("Asset ID: {}".format(scrutinized_asset['asset-id']))
        print(json.dumps(scrutinized_asset, indent=4))
        print("URL: {}".format(account_info['created-assets'][0]['params']['url']))
        break

{'address': 'I5XCXAH33IQAWY574KUM3R3OZNSDMWIWB5WBZ7FXPHNZCWBBQD5F6LYR4A', 'amount': 4999000, 'amount-without-pending-rewards': 4999000, 'apps-local-state': [], 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0}, 'assets': [{'amount': 1, 'asset-id': 121850449, 'is-frozen': False}], 'created-apps': [], 'created-assets': [{'index': 121850449, 'params': {'creator': 'I5XCXAH33IQAWY574KUM3R3OZNSDMWIWB5WBZ7FXPHNZCWBBQD5F6LYR4A', 'decimals': 0, 'default-frozen': False, 'manager': 'I5XCXAH33IQAWY574KUM3R3OZNSDMWIWB5WBZ7FXPHNZCWBBQD5F6LYR4A', 'metadata-hash': 'dtbFu/HZOl7mEmR0an1OrDN4vMo2SQeu1690lqNwzOs=', 'name': "Alice's Artwork 001", 'name-b64': 'QWxpY2UncyBBcnR3b3JrIDAwMQ==', 'total': 1, 'unit-name': 'ALICE001', 'unit-name-b64': 'QUxJQ0UwMDE=', 'url': 'https://path/to/my/asset/details', 'url-b64': 'aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM='}}], 'min-balance': 200000, 'pending-rewards': 0, 'reward-base': 27521, 'rewards': 0, 'round': 25440549, 'status': 'Offline', 'total-apps-opted-